In [1]:
import os
import shutil
import pandas as pd

In [2]:
# dataDicts = []
# for root, dirs, files in os.walk('oasis3downloads'):
#     for file in files:
#         if 'nii.gz' in file:
#             filepath = os.path.join(root,file)
#             if '_MR_' in filepath:
#                 type = str(filepath).split('_')[-1]
#                 type = type.split('.nii.gz')[0]
#                 baseFolder = str(filepath).split(os.sep)[1]
#                 newPath = os.path.join('NIFTI',baseFolder,file)
#                 temp = pd.Series()
#                 temp = {'origpath':filepath, 'type':type, 'newpath':newPath}
#                 dataDicts.append(temp)
#             break

In [3]:
df = pd.read_csv('Oasis3_testLabels.txt',names=['ID','label','extra'],sep='\t',low_memory=False,dtype={'ID':str,'label':int,'extra':int})
df

,ID,label,extra
0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0
1,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS309...,0,0
2,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0
3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0
4,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0
...,...,...,...
54370,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0
54371,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS309...,0,0
54372,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS302...,0,0
54373,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS302...,0,0


In [4]:
def extractType(df):
    ID = df['ID']
    ID = ID.split('__s')[0]
    type = ID.split('_')[-1]
    return type

In [5]:
df['type'] = df.apply(extractType,axis=1)

In [6]:
print(df['type'].unique())

['T1w' 'T2star' 'fieldmap' 'dwi' 'swi' 'minIP' 'T2w' 'CT' 'asl' 'bold'
 'FLAIR' 'angio' 'GRE' 'pet' 'pasl']


In [7]:
df['description']=df['type']

In [8]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"FS":4,"PD":5,
             "SWI":6, "GRE":7, "T2*":8, "DWI":9, "ADC":10, "BOLD":11,
             "FlowSensitve":12,"PWI":13,"ASL":14,"Hippo":15,
             "DTI":16,"Other":17}
def assignLabel(dataframe):
    description = str(dataframe['description'])
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP']
    if description in otherCategories:
        return 17
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 5
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='FS':
        return 4
    if description =='pd':
        return 5
    if description =='swi':
        return 6
    if description.lower() =='gre': #from oasis3 can be magnitude or phase but no more description
        return 7
    if description =='T2star':
        return 8
    if description =='dwi':
        return 9
    if description =='adc':
        return 10
    if description =='bold':
        return 11
    if description =='angio': #flowsensitive
        return 12
    if description =='pwi' or description=='cbf':
        return 13
    if 'asl' in description:
        return 14
    if description == 'hippo':
        return 15
    if description == 'dti':
        return 16
    return -1

In [9]:
df['label'] = df.apply(assignLabel,axis=1)

In [10]:
df['label'].value_counts()

label
17    14075
11     8275
2      6350
0      5750
9      5400
8      3625
14     2700
7      2400
6      2025
3      2025
12     1750
Name: count, dtype: int64

In [11]:
df[df['label']==-1]

,ID,label,extra,type,description


In [12]:
df = df[df['label']!=-1]

In [13]:
df = df[['ID','label','extra']]

In [15]:
df.to_csv('Oasis3_testLabels_extensive2.txt',index=False, header=False, sep='\t')